In [7]:
import torch
from torch.nn.parameter import Parameter
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch.nn.init as init
from torch.autograd import Variable
import torch.utils.data as Data
from simulation import *
from model_function import *
from evaluation_utils import *
from sklearn.model_selection import train_test_split
import csv
import time

In [5]:
def run_sim(res):
    expression = res[1]
    partition = res[2]
    partition[np.where(partition > 1)] = 0
    feature_meta = res[3]
    feature_meta_true = res[4]
    true_meta = res[5]

    # split into training and testing
    target = expression[:, expression.shape[1]-1]
    data = expression[:,0:(expression.shape[1]-1)]
    x_train, x_test, y_train, y_test = train_test_split(
            data,target,test_size = 0.3,random_state = 0)

    sparsify_coefficient = 0.3
    threshold_layer_size = 100

    # set the framework of the predicting net
    num_hidden_layer_neuron_list = [20] 

    features_num = expression.shape[0]

    sparsify_hidden_layer_size_dict = getLayerSizeList(partition, threshold_layer_size, sparsify_coefficient)
    degree_dict = getNodeDegreeDict(partition)
    partition_mtx_dict = getPartitionMatricesList(sparsify_hidden_layer_size_dict, degree_dict, feature_meta, partition)

    keep_prob = 0.3

    import warnings
    import csv
    count = 0
    while (count<10):
        net = Net(partition_mtx_dict, num_hidden_layer_neuron_list, keep_prob).to("cuda")

        BATCH_SIZE = 8

        warnings.filterwarnings("ignore", category=UserWarning)

        torch.manual_seed=68

        optimizer = torch.optim.Adam(net.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08)  #

        loss_func = torch.nn.CrossEntropyLoss()

        x = (torch.from_numpy(x_train)).type(torch.FloatTensor)
        y = (torch.from_numpy(y_train)).type(torch.LongTensor)

        torch_dataset = Data.TensorDataset(x, y)

        loader = Data.DataLoader(
            dataset=torch_dataset,    
            batch_size=BATCH_SIZE,    
            shuffle=True,             
            num_workers=0           
        )

        x = Variable(x)
        y = Variable(y)

        acc_train = []
        acc_test = []

        for epoch in range(100): 
            net.train()
            if epoch%10 == 0:
                optimizer.param_groups[0]['lr'] *= 0.95

            for step, (x_batch, y_batch) in enumerate(loader):
                x_batch = x_batch.to('cuda')
                y_batch = y_batch.to('cuda')
                optimizer.zero_grad()
                prediction = net(x_batch).to('cuda')
                loss = loss_func(prediction,y_batch)
                loss.backward()
                optimizer.step()

            with torch.no_grad():
                x = x.to('cuda')
                y = y.to('cuda')
                prediction = net(x).to('cuda')
                pred_y = prediction.cpu().data.numpy().squeeze()
                target_y = y.cpu().data.numpy()
                accuracy = sum(target_y ==np.argmax(pred_y, axis=1))/len(target_y) 
                acc_train.append(accuracy)

                test_input_tensor = (torch.from_numpy(x_test)).type(torch.FloatTensor).to('cuda')
                out_probs = net(test_input_tensor).cpu().data.numpy().squeeze()
                out_classes = np.argmax(out_probs, axis=1)
                acc_test.append(sum(out_classes == y_test) / len(y_test))
        

        if np.mean(acc_train[-10:-1])>0.6:
            count += 1
            train_mean = np.mean(acc_train[-10:-1])
            test_mean = np.mean(acc_test[-10:-1])

            degree = np.sum((partition>0),0)+1
            count_feature = (np.sum(feature_meta, 0)+1)

            res = evalution(net, feature_meta_true, true_meta, degree, count_feature)

            with open("sim_result.csv","a") as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow([n_feature, n_meta, N, multi_rate, poisson_rate, effective_ratio] + [train_mean, test_mean] + res)

In [4]:
with open("sim_result.csv","a") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["n_feature", "n_meta", "N", "multi_rate", "poisson_rate", "effective_ratio"] + ["train_acc", "test_acc", "rate", "feature_auc", "feature_prauc", "meta_auc", "meta_prauc", "degree_auc", 'degree_prauc'])

In [ ]:
n_meta = 2000; n_feature = 1000; max_meta_link = 3; max_feature_link = 3; N = 500; link = 'logistic'; ba_m = 2; 
ba_power = 0.5; cov_base = 0.6

for n_meta in [2000,1000]: #[1000, 1500, 2000]
    for multi_rate in [0.5]:
        for poisson_rate in [3,2,1]: #[1, 2, 3]
            for effective_ratio in [0.05, 0.1]: #[0.05, 0.1]
                for N in [100, 500, 1000, 2000, 4000]: ##500, 1000, 2000, 5000
                    res = matching_simulation(n_feature, n_meta, N, ba_m, ba_power, multi_rate, poisson_rate, max_meta_link, max_feature_link, effective_ratio, link, cov_base)
                    run_sim(res)

Finish generating Y with 1 class 50, 0 class 50


/home/leqi/leqi/code/sparse_nn/pytorch_func2.py:167: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370172916/work/aten/src/ATen/native/IndexingUtils.h:25.)
  self.linear.weight.data[self.mask] = 0 # zero out bad weights
/home/leqi/leqi/code/sparse_nn/simulation_2.py:120: RuntimeWarning: covariance is not positive-semidefinite.
  X = np.random.multivariate_normal(mean, covariance, N)


Finish generating Y with 1 class 250, 0 class 250


/home/leqi/leqi/code/sparse_nn/simulation_2.py:120: RuntimeWarning: covariance is not positive-semidefinite.
  X = np.random.multivariate_normal(mean, covariance, N)


Finish generating Y with 1 class 500, 0 class 500
